
# Task:

* Take any csv/text file of 2+ GB of your choice. --- (You can do this assignment on Google colab)

* Read the file ( Present approach of reading the file )

* Try different methods of file reading eg: Dask, Modin, Ray, pandas and present your findings in term of computational     efficiency

* Perform basic validation on data columns : eg: remove special character , white spaces from the col name

* As you already know the schema hence create a YAML file and write the column name in YAML file. --define separator of   
  read and write file, column name in YAML

* Validate number of columns and column name of ingested file with YAML.

* Write the file in pipe separated text file (|) in gz format.

* Create a summary of the file:

    Total number of rows,

    total number of columns

    file size
    




## Read dataset with different methods

In [1]:
import os
import time
import warnings
import gc
warnings.filterwarnings("ignore")

In [2]:
#Size of the file
os.path.getsize('../input/amazon-books-reviews/Books_rating.csv')


2859504349

### Read dataset with Dask

In [3]:
from dask import dataframe as dd
start = time.time()
dask_df = dd.read_csv('../input/amazon-books-reviews/Books_rating.csv')
end = time.time()
print("Read csv with dask: ",(end-start),"sec")

Read csv with dask:  0.05772757530212402 sec


### Read dataset with Pandas 

In [4]:
import pandas as pd
start = time.time()
df = pd.read_csv('../input/amazon-books-reviews/Books_rating.csv')
end = time.time()
print("Read csv with pandas: ",(end-start),"sec")

Read csv with pandas:  93.29925584793091 sec


### Read dataset with Modin and Ray

In [5]:
!pip install modin
!pip install ray 

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 956.2/956.2 kB 2.3 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 761.5/761.5 kB 17.1 MB/s eta 0:00:0000:01


In [29]:
import modin.pandas as pd
import ray
ray.shutdown()
ray.init()
start = time.time()
df = pd.read_csv('../input/amazon-books-reviews/Books_rating.csv')
end = time.time()
print("Read csv with modin and ray: ",(end-start),"sec")
ray.shutdown()

2022-10-10 04:45:00,451	INFO worker.py:1518 -- Started a local Ray instance.
(raylet) [2022-10-10 04:45:10,345 E 995 1013] (raylet) file_system_monitor.cc:105: /tmp/ray/session_2022-10-10_04-44-56_086358_17 is over 95% full, available space: 157056401408; capacity: 4327869423616. Object creation will fail if spilling is required.
(raylet) [2022-10-10 04:45:20,355 E 995 1013] (raylet) file_system_monitor.cc:105: /tmp/ray/session_2022-10-10_04-44-56_086358_17 is over 95% full, available space: 157056364544; capacity: 4327869423616. Object creation will fail if spilling is required.
(raylet) [2022-10-10 04:45:30,367 E 995 1013] (raylet) file_system_monitor.cc:105: /tmp/ray/session_2022-10-10_04-44-56_086358_17 is over 95% full, available space: 157056364544; capacity: 4327869423616. Object creation will fail if spilling is required.


Read csv with modin and ray:  35.61858630180359 sec


Here Dask is better than Pandas, Modin and Ray, with the reading time of  0.058 sec

In [6]:
gc.collect()
from dask import dataframe as dd
df = dd.read_csv('../input/amazon-books-reviews/Books_rating.csv',delimiter=',')

In [7]:
df.info()

<class 'dask.dataframe.core.DataFrame'>
Columns: 10 entries, Id to review/text
dtypes: object(6), float64(2), int64(2)

In [8]:
#No, of Columns
len(df.columns)

10

In [9]:
# remove special character
df.columns=df.columns.str.replace('[#,@,&]','')

In [10]:
#To remove white space from columns
df.columns = df.columns.str.replace(' ', '')

In [35]:
df.columns


Index(['Id', 'Title', 'Price', 'User_id', 'profileName', 'review/helpfulness',
       'review/score', 'review/time', 'review/summary', 'review/text'],
      dtype='object')

## Write YAML file & Validation

In [11]:
%%writefile testutility.py
import logging
import os
import subprocess
import yaml
import pandas as pd
import datetime 
import gc
import re


################
# File Reading #
################

def read_config_file(filepath):
    with open(filepath, 'r') as stream:
        try:
            return yaml.safe_load(stream)
        except yaml.YAMLError as exc:
            logging.error(exc)


def replacer(string, char):
    pattern = char + '{2,}'
    string = re.sub(pattern, char, string) 
    return string

def col_header_val(df,table_config):
    '''
    replace whitespaces in the column
    and standardized column names
    '''
    df.columns = df.columns.str.lower()
    df.columns = df.columns.str.replace('[^\w]','_',regex=True)
    df.columns = list(map(lambda x: x.strip('_'), list(df.columns)))
    df.columns = list(map(lambda x: replacer(x,'_'), list(df.columns)))
    expected_col = list(map(lambda x: x.lower(),  table_config['columns']))
    expected_col.sort()
    df.columns =list(map(lambda x: x.lower(), list(df.columns)))
    df = df.reindex(sorted(df.columns), axis=1)
    if len(df.columns) == len(expected_col) and list(expected_col)  == list(df.columns):
        print("column name and column length validation passed")
        return 1
    else:
        print("column name and column length validation failed")
        mismatched_columns_file = list(set(df.columns).difference(expected_col))
        print("Following File columns are not in the YAML file",mismatched_columns_file)
        missing_YAML_file = list(set(expected_col).difference(df.columns))
        print("Following YAML columns are not in the file uploaded",missing_YAML_file)
        logging.info(f'df columns: {df.columns}')
        logging.info(f'expected columns: {expected_col}')
        return 0

Writing testutility.py


### Write YAML file

In [12]:
%%writefile file.yaml
file_type: csv
dataset_name: testfile
file_name: Books_rating
table_name: edsurv
inbound_delimiter: ","
outbound_delimiter: "|"
skip_leading_rows: 1
columns: 
    - Id
    - Title
    - Price
    - User_id
    - profileName
    - review/helpfulness
    - review/score
    - review/time
    - review/summary
    - review/text

Writing file.yaml


In [13]:
# Read config file
import testutility as util
config_data = util.read_config_file("file.yaml")

In [14]:
config_data['inbound_delimiter']

','

In [15]:
#inspecting data of config file
config_data

{'file_type': 'csv',
 'dataset_name': 'testfile',
 'file_name': 'Books_rating',
 'table_name': 'edsurv',
 'inbound_delimiter': ',',
 'outbound_delimiter': '|',
 'skip_leading_rows': 1,
 'columns': ['Id',
  'Title',
  'Price',
  'User_id',
  'profileName',
  'review/helpfulness',
  'review/score',
  'review/time',
  'review/summary',
  'review/text']}

In [45]:
# Normal reading process of the file with dask
import pandas as pd
df_sample = pd.read_csv('../input/amazon-books-reviews/Books_rating.csv',delimiter=',')
df_sample.head()

,Id,Title,Price,User_id,profileName,review/helpfulness,review/score,review/time,review/summary,review/text
0,1882931173,Its Only Art If Its Well Hung!,NaN,AVCGYZL8FQQTD,"Jim of Oz ""jim-of-oz""",7/7,4.0,940636800,Nice collection of Julie Strain images,This is only for Julie Strain fans. It's a col...
1,0826414346,Dr. Seuss: American Icon,NaN,A30TK6U7DNS82R,Kevin Killian,10/10,5.0,1095724800,Really Enjoyed It,I don't care much for Dr. Seuss but after read...
2,0826414346,Dr. Seuss: American Icon,NaN,A3UH4UZ4RSVO82,John Granger,10/11,5.0,1078790400,Essential for every personal and Public Library,"If people become the books they read and if ""t..."
3,0826414346,Dr. Seuss: American Icon,NaN,A2MVUWT453QH61,"Roy E. Perry ""amateur philosopher""",7/7,4.0,1090713600,Phlip Nel gives silly Seuss a serious treatment,"Theodore Seuss Geisel (1904-1991), aka &quot;D..."
4,0826414346,Dr. Seuss: American Icon,NaN,A22X4XUPKF66MR,"D. H. Richards ""ninthwavestore""",3/3,4.0,1107993600,Good academic overview,Philip Nel - Dr. Seuss: American IconThis is b...


In [52]:
# read the file using config file
file_type = config_data['file_type']
source_file = "../input/amazon-books-reviews/" + config_data['file_name'] + f'.{file_type}'
#print("",source_file)
df = pd.read_csv(source_file,config_data['inbound_delimiter'])
df.head()

,Id,Title,Price,User_id,profileName,review/helpfulness,review/score,review/time,review/summary,review/text
0,1882931173,Its Only Art If Its Well Hung!,NaN,AVCGYZL8FQQTD,"Jim of Oz ""jim-of-oz""",7/7,4.0,940636800,Nice collection of Julie Strain images,This is only for Julie Strain fans. It's a col...
1,0826414346,Dr. Seuss: American Icon,NaN,A30TK6U7DNS82R,Kevin Killian,10/10,5.0,1095724800,Really Enjoyed It,I don't care much for Dr. Seuss but after read...
2,0826414346,Dr. Seuss: American Icon,NaN,A3UH4UZ4RSVO82,John Granger,10/11,5.0,1078790400,Essential for every personal and Public Library,"If people become the books they read and if ""t..."
3,0826414346,Dr. Seuss: American Icon,NaN,A2MVUWT453QH61,"Roy E. Perry ""amateur philosopher""",7/7,4.0,1090713600,Phlip Nel gives silly Seuss a serious treatment,"Theodore Seuss Geisel (1904-1991), aka &quot;D..."
4,0826414346,Dr. Seuss: American Icon,NaN,A22X4XUPKF66MR,"D. H. Richards ""ninthwavestore""",3/3,4.0,1107993600,Good academic overview,Philip Nel - Dr. Seuss: American IconThis is b...


In [53]:
#validate the header of the file
util.col_header_val(df,config_data)

column name and column length validation failed
Following File columns are not in the YAML file ['review_helpfulness', 'review_time', 'review_text', 'review_score', 'review_summary']
Following YAML columns are not in the file uploaded ['review/time', 'review/text', 'review/score', 'review/helpfulness', 'review/summary']


0

In [54]:
print("columns of files are:" ,df.columns)
print("columns of YAML are:" ,config_data['columns'])

columns of files are: Index(['id', 'title', 'price', 'user_id', 'profilename', 'review_helpfulness',
       'review_score', 'review_time', 'review_summary', 'review_text'],
      dtype='object')
columns of YAML are: ['Id', 'Title', 'Price', 'User_id', 'profileName', 'review/helpfulness', 'review/score', 'review/time', 'review/summary', 'review/text']


In [55]:
if util.col_header_val(df,config_data)==0:
    print("validation failed")
    # write code to reject the file
else:
    print("col validation passed")
    # write the code to perform further action
    # in the pipleine

column name and column length validation failed
Following File columns are not in the YAML file ['review_helpfulness', 'review_time', 'review_text', 'review_score', 'review_summary']
Following YAML columns are not in the file uploaded ['review/time', 'review/text', 'review/score', 'review/helpfulness', 'review/summary']
validation failed


In [17]:
df.dtypes

Id                      int64
Title                  object
Price                 float64
User_id                object
profileName            object
review/helpfulness     object
review/score          float64
review/time             int64
review/summary         object
review/text            object
dtype: object

In [18]:
df = pd.read_csv('../input/amazon-books-reviews/Books_rating.csv',delimiter=',')

In [19]:
df.dtypes

Id                     object
Title                  object
Price                 float64
User_id                object
profileName            object
review/helpfulness     object
review/score          float64
review/time             int64
review/summary         object
review/text            object
dtype: object

In [21]:
import datetime
import csv
import gzip

#df = pd.read_csv('../input/amazon-books-reviews/Books_rating.csv',delimiter=',')

# Write csv in gz format in pipe separated text file (|)
df.to_csv('/kaggle/working/books_rating.csv.gz',
          sep='|',
          header=True,
          index=False,
          compression='gzip')

In [22]:
gc.collect()


2841

In [26]:
#size of the gz format folder
os.path.getsize('/kaggle/working/books_rating.csv.gz')

1058085693